In [755]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [1170]:
df = pd.read_csv('default_data_all_variables.csv')
target = pd.read_csv('targets.csv')

In [1163]:
df = df.loc[:, ~df.columns.str.contains('HomeTeam|AwayTeam')]

In [1090]:
df[['home_win_odds','draw_odds','away_win_odds']] = target[['home_win_odds','draw_odds','away_win_odds']]

In [1166]:
target = target[['FTHG', 'FTAG']]

In [1168]:
X_train = df[:-1000]
y_train = target[:-1000]
X_val = df[-1000:-500]
y_val = target[-1000:-500]
X_test = df[-500:]
y_test = target[-500:]


In [1172]:
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
# Define the optimizer with a specific learning rate
optimizer = Adam(learning_rate=0.0005)
model = Sequential()

# Input layer
model.add(Dense(256, activation='relu', input_shape=(177,)))  # 278 is the number of input features
# model.add(Dropout(0.1))
# Hidden layers
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.1))  # Optional, to prevent overfitting
# model.add(Dense(138, activation='relu'))
# model.add(Dropout(0.1))  # Optional, to prevent overfitting
model.add(Dense(48, activation='relu'))

# Output layer (with softmax to ensure sum-to-1 constraint)
model.add(Dense(2, activation='relu'))

# Compile the model
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10)
class_weights = {'home_win_odds': 1.0, 'draw_odds': 1, 'away_win_odds': 1.0}
# Train the model
history = model.fit(X_train, y_train, epochs=400, batch_size=1000, validation_data=(X_val, y_val), class_weight=class_weights, callbacks=[early_stopping])
# , callbacks=[early_stopping]
# Evaluate the model
results = model.evaluate(X_test, y_test)

Epoch 1/400


C:\Users\ciesl\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 2.4732 - mae: 1.1475 - val_loss: 1.6517 - val_mae: 1.0518
Epoch 2/400
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.6371 - mae: 0.9735 - val_loss: 1.5567 - val_mae: 1.0081
Epoch 3/400
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.5429 - mae: 0.9558 - val_loss: 1.4681 - val_mae: 0.9513
Epoch 4/400
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.4804 - mae: 0.9296 - val_loss: 1.4356 - val_mae: 0.9437
Epoch 5/400
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.4339 - mae: 0.9201 - val_loss: 1.4250 - val_mae: 0.9286
Epoch 6/400
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.4096 - mae: 0.9097 - val_loss: 1.4092 - val_mae: 0.9204
Epoch 7/400
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.3990 - mae: 0.9054 - val_loss: 1.4087 - val_mae: 0.9281
Epoch 8/400
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.3791 - mae: 0.9035 - val_loss: 1.3945 - val_mae: 0.9160
Epoch 9/400
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.3726 - mae: 0.

In [1174]:
results

[1.4660910367965698, 0.9435939192771912]

In [1176]:
y_pred = model.predict(X_test)

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


In [1178]:
def choose_value2(row, val_draw):
    if row['FTHG']-val_draw > row['FTAG']:
        return 'H'
    elif np.abs(row['FTHG']- row['FTAG']) <val_draw:
        return 'D'
    else:
        return 'A'

In [1180]:
trg = target[-500:]
trg['result'] = trg.apply(choose_value2, val_draw = 0.2, axis=1)
trg.reset_index(inplace = True, drop = True)

C:\Users\ciesl\AppData\Local\Temp\ipykernel_38164\530522862.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trg['result'] = trg.apply(choose_value2, val_draw = 0.2, axis=1)


In [1182]:
res = pd.DataFrame(y_pred, columns = ['FTHG', 'FTAG'])
res['result'] = res.apply(choose_value2, val_draw = 0.2, axis=1)
matches = res['result'] == trg['result']

In [1184]:
res

,FTHG,FTAG,result
0,1.697887,1.117448,H
1,1.951991,1.114648,H
2,2.386582,0.657505,H
3,1.658149,0.973619,H
4,1.504676,1.183390,H
...,...,...,...
495,1.373233,1.357486,D
496,1.252389,0.919517,H
497,1.556953,1.884906,A
498,1.051635,1.085481,D


In [1186]:
percentage_compatibility = matches.mean() * 100
percentage_compatibility

48.0

In [1188]:
trg['res2'] = res['result']

C:\Users\ciesl\AppData\Local\Temp\ipykernel_38164\512750724.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trg['res2'] = res['result']


In [1190]:
trg

,FTHG,FTAG,home_win_odds,draw_odds,away_win_odds,result,res2
0,1,1,0.458753,0.266787,0.274460,D,H
1,1,1,0.628708,0.196715,0.174577,D,H
2,5,1,0.775290,0.136685,0.088025,H,H
3,2,1,0.481468,0.250448,0.268085,H,H
4,1,0,0.407562,0.271064,0.321374,H,H
...,...,...,...,...,...,...,...
495,3,2,0.392395,0.292396,0.315209,H,D
496,2,0,0.486209,0.284551,0.229241,H,H
497,4,0,0.398873,0.273154,0.327973,H,A
498,1,0,0.533203,0.245773,0.221023,H,D


In [1192]:
def money_won(row):
    if row['res2'] == row['result']:
        if row['res2'] == 'H':
            return 1/row['home_win_odds']*0.88/1.05
        elif row['res2'] == 'D':
            return 1/row['draw_odds']*0.88/1.05
        else:
             return 1/row['away_win_odds']*0.88/1.05
    else:
        return 0

In [1194]:
trg['money_won'] = trg.apply(money_won, axis=1)

C:\Users\ciesl\AppData\Local\Temp\ipykernel_38164\32439853.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trg['money_won'] = trg.apply(money_won, axis=1)


In [1196]:
trg['money_won'].sum()

429.46699737020276

In [305]:
import xgboost as xgb
model = xgb.XGBRegressor(n_estimators = 200, max_depth = 30)
model.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=30, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [306]:
predictions = model.predict(X_test)

predictions

array([[ 0.8270622 ,  2.3593216 ],
       [ 2.248635  ,  1.0351251 ],
       [ 1.5020763 ,  1.4628187 ],
       [ 1.312572  ,  2.843878  ],
       [ 1.8918837 ,  1.5118762 ],
       [ 1.1661514 ,  1.1618774 ],
       [ 1.7009051 ,  1.3253386 ],
       [ 3.359922  ,  0.7836386 ],
       [ 0.3275212 ,  0.87447226],
       [ 1.4420677 ,  1.0026604 ],
       [ 2.0355852 ,  2.23652   ],
       [ 1.516636  ,  0.83263195],
       [ 1.132068  ,  0.6848485 ],
       [ 0.12954983,  1.7100691 ],
       [ 1.1620331 ,  0.9821303 ],
       [ 1.6714836 ,  1.3314221 ],
       [ 2.91517   ,  0.2859898 ],
       [ 1.3114005 ,  1.7994162 ],
       [ 0.8185517 ,  0.6334857 ],
       [ 1.4781234 ,  4.00139   ],
       [ 1.326266  ,  0.83327425],
       [ 1.1420169 ,  1.980266  ],
       [ 1.3939782 ,  2.5562642 ],
       [ 1.7294489 ,  1.3665495 ],
       [ 1.860204  ,  2.9007995 ],
       [ 1.0119709 ,  1.4083009 ],
       [ 2.183454  ,  1.2615047 ],
       [ 2.2126763 ,  1.8337733 ],
       [ 1.6929687 ,

In [307]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, predictions)

1.022264981083572

In [ ]:
pred_df = pd.DataFrame(predictions, columns = ['home_win_odds', 'draw_odds', 'away_win_odds'])
pred_df[['FTHG', 'FTAG']] = trg[['FTHG', 'FTAG']]
pred_df['probability converted'] = pred_df.apply(choose_value, axis=1)
pred_df['probability converted'].sum()

In [ ]:
import matplotlib.pyplot as plt
feature_importances = model.feature_importances_
feature_names = X_train.columns

# Create a DataFrame for easier manipulation
feature_importances_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
})

# Sort by importance and get the top 20 features
top_20_features = feature_importances_df.sort_values(by='Importance', ascending=False).head(20)

# Plot the top 20 feature importances
plt.figure(figsize=(10, 6))
plt.barh(top_20_features['Feature'], top_20_features['Importance'], color='skyblue')
plt.xlabel('Importance Score')
plt.ylabel('Features')
plt.title('Top 20 Feature Importances')
plt.gca().invert_yaxis()  # Invert y-axis to have the most important feature at the top
plt.show()

In [971]:
target[-150:]

,FTHG,FTAG,home_win_odds,draw_odds,away_win_odds
5900,2,0,0.134539,0.193082,0.672379
5901,2,3,0.424772,0.260270,0.314958
5902,0,4,0.143382,0.193517,0.663101
5903,4,0,0.179575,0.213786,0.606640
5904,1,1,0.418944,0.283305,0.297751
...,...,...,...,...,...
6045,1,1,0.712907,0.170303,0.116789
6046,1,1,0.592215,0.217852,0.189933
6047,2,1,0.385471,0.240114,0.374415
6048,0,3,0.253574,0.227835,0.518591


In [1132]:
trg = target[-500:]

In [1134]:
def choose_value(row):
    if row['FTHG'] > row['FTAG']:
        return 'H'
    elif row['FTHG'] == row['FTAG']:
        return 'D'
    else:
        return 'A'

# Apply the function to each row
trg['result'] = trg.apply(choose_value, axis=1)

C:\Users\ciesl\AppData\Local\Temp\ipykernel_38164\1274032490.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trg['result'] = trg.apply(choose_value, axis=1)


In [1136]:
trg['result']

18547    D
18548    D
18549    H
18550    H
18551    H
        ..
19042    H
19043    H
19044    H
19045    H
19046    A
Name: result, Length: 500, dtype: object